<a href="https://colab.research.google.com/github/cloudpendyala/pendyala_ai/blob/main/FunctionCalling/FunctionCallingAndDataExtractionUsingLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install openai python-dotenv


In [65]:
# Import necessary libraries
from openai import OpenAI
import json
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the environment variable
openai_api_key = os.getenv('OPENAI_API_KEY')

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=openai_api_key)

# Define a function to calculate the area of a circle
def calculate_area_of_circle(radius):
    """Calculate the area of a circle given its radius"""
    import math
    area = math.pi * (radius ** 2)
    return json.dumps({"radius": radius, "area": area})

# Define a function to calculate the area of a cube
def calculate_area_of_cube(length):
    """Calculate the area of a cube given its length"""
    area = 6 * (length ** 2)
    return json.dumps({"length": length, "area": area})

def run_circle_area_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the area of a circle with a radius of 5 and cube with length 12 metres?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate_area_of_circle",
                "description": "Calculate the area of a circle given its radius",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "radius": {
                            "type": "number",
                            "description": "The radius of the circle",
                        }
                    },
                    "required": ["radius"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "calculate_area_of_cube",
                "description": "Calculate the area of a cube given its length",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "length": {
                            "type": "number",
                            "description": "The length of the cube",
                        }
                    },
                    "required": ["length"],
                },
            }
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        available_functions = {
            "calculate_area_of_circle": calculate_area_of_circle,
            "calculate_area_of_cube": calculate_area_of_cube
        }
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            # Call the function with appropriate arguments
            if function_name == "calculate_area_of_circle":
                function_response = function_to_call(
                    radius=function_args.get("radius")
                )
            elif function_name == "calculate_area_of_cube":
                function_response = function_to_call(
                    length=function_args.get("length")
                )

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
    return "No function calls were needed."

# Run the conversation and print the result
circle_area_conversation = run_circle_area_conversation()

# Print the content of the final response message
if isinstance(circle_area_conversation, str):
    print(circle_area_conversation)
else:
    print(circle_area_conversation.choices[0].message.content)



The area of a circle with a radius of 5 meters is approximately 78.54 square meters. 

The surface area of a cube with a side length of 12 meters is 864 square meters.


In [63]:
# Import necessary libraries
from openai import OpenAI
import json
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the environment variable
openai_api_key = os.getenv('OPENAI_API_KEY')

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=openai_api_key)

# Define a function to convert Celsius to Fahrenheit
def celsius_to_fahrenheit(celsius):
    """Convert Celsius to Fahrenheit"""
    fahrenheit = (celsius * 9/5) + 32
    return json.dumps({"celsius": celsius, "fahrenheit": fahrenheit})

# Define a function to convert Fahrenheit to Celsius
def fahrenheit_to_celsius(fahrenheit):
    """Convert Fahrenheit to Celsius"""
    celsius = (fahrenheit - 32) * 5/9
    return json.dumps({"fahrenheit": fahrenheit, "celsius": celsius})

def run_temperature_conversion_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the temperature in Fahrenheit if it's 25 degrees Celsius? and what's the 80 Fahrenheit in celsius"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "celsius_to_fahrenheit",
                "description": "Convert Celsius to Fahrenheit",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "celsius": {
                            "type": "number",
                            "description": "The temperature in Celsius",
                        }
                    },
                    "required": ["celsius"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "fahrenheit_to_celsius",
                "description": "Convert Fahrenheit to Celsius",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "fahrenheit": {
                            "type": "number",
                            "description": "The temperature in Fahrenheit"
                        }
                    },
                    "required": ["fahrenheit"]
                }
            }
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

    # Extracting the message and tool calls safely
    if response.choices and response.choices[0].message:
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
    else:
        return "No valid response from the model."

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        available_functions = {
            "celsius_to_fahrenheit": celsius_to_fahrenheit,
            "fahrenheit_to_celsius": fahrenheit_to_celsius
        }
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            # Call the function with appropriate arguments
            if function_name == "celsius_to_fahrenheit":
                function_response = function_to_call(celsius=function_args.get("celsius"))
            elif function_name == "fahrenheit_to_celsius":
                function_response = function_to_call(fahrenheit=function_args.get("fahrenheit"))

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response

        # Get a new response from the model where it can see the function response
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        if second_response.choices and second_response.choices[0].message:
            return second_response.choices[0].message.content
        else:
            return "No valid response from the model after function call."
    return "No function calls were needed."

# Run the conversation and print the result
temperature_conversion = run_temperature_conversion_conversation()

# Print the content of the final response message
print(temperature_conversion)


If it's 25 degrees Celsius, that translates to 77 degrees Fahrenheit. Conversely, 80 degrees Fahrenheit is approximately 26.67 degrees Celsius.
